# Project on Finding the Best Location for Opening a Chinese Restaurant in Manhattan, United State

**Download all the dependencies that we will need.**

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: d

**Download the dataset.**

In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


**Load the data.**

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

**All the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.**

In [4]:
neighborhoods_data = newyork_data['features']

**Transform this data of nested Python dictionaries into a pandas dataframe.**

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

**Then let's loop through the data and fill the dataframe one row at a time.**

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

**Create a new dataframe of the Manhattan data.**

In [7]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


**Get the geographical coordinates of Manhattan.**

In [8]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


**Visualise the neighbourhoods in Manhattan.**

In [9]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

**Define Foursquare Credentials and Version. (Client ID and client secret will not be shown here.)**

In [10]:
CLIENT_ID =  # your Foursquare ID
CLIENT_SECRET = # your Foursquare Secret
VERSION = '20201221' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

**Create a function to find all nearby venues for all the neighborhoods in Manhattan.**

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**Run the above function on each neighborhood and create a new dataframe called _manhattan_venues_.**

In [12]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


**Analyse each neighbourhood.**

In [13]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skat

**Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.**

In [14]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

**Write a function to sort the venues in descending order.**

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Create the new dataframe and display the top 10 venues for each neighborhood.**

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Memorial Site,Clothing Store,Gym,Beer Garden,Shopping Mall,Burger Joint,Boat or Ferry
1,Carnegie Hill,Coffee Shop,Café,Bar,Pizza Place,Yoga Studio,Wine Shop,Cosmetics Shop,Bookstore,Gym / Fitness Center,Gym
2,Central Harlem,Seafood Restaurant,French Restaurant,African Restaurant,American Restaurant,Chinese Restaurant,Bar,Gym / Fitness Center,Ethiopian Restaurant,Grocery Store,Park
3,Chelsea,Coffee Shop,Bakery,Art Gallery,American Restaurant,Hotel,Wine Shop,Italian Restaurant,French Restaurant,Cupcake Shop,Cycle Studio
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Salon / Barbershop,Spa,Optical Shop,Dessert Shop,Asian Restaurant,Bar


**Run k-means to cluster the neighborhood into 5 clusters.**

In [17]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 0, 2, 0, 2, 2, 3, 0, 2], dtype=int32)

**Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.**

In [18]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Discount Store,Gym,Sandwich Place,Coffee Shop,Video Game Store,Tennis Stadium,Seafood Restaurant,Donut Shop,Supplement Shop,Pharmacy
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Salon / Barbershop,Spa,Optical Shop,Dessert Shop,Asian Restaurant,Bar
2,Manhattan,Washington Heights,40.851903,-73.936900,3,Café,Bakery,Bank,Mobile Phone Shop,Grocery Store,Wine Shop,Supermarket,Sandwich Place,Park,Chinese Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,3,Mexican Restaurant,Restaurant,Café,Lounge,Frozen Yogurt Shop,Bakery,Pizza Place,Park,Chinese Restaurant,Caribbean Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,3,Pizza Place,Mexican Restaurant,Deli / Bodega,Coffee Shop,Café,Yoga Studio,Bakery,Latin American Restaurant,Park,Cocktail Bar


**Finally, visualise the resulting clusters.**

In [19]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Examine clusters.**

**Cluster 1**

In [20]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Salon / Barbershop,Spa,Optical Shop,Dessert Shop,Asian Restaurant,Bar
6,Central Harlem,Seafood Restaurant,French Restaurant,African Restaurant,American Restaurant,Chinese Restaurant,Bar,Gym / Fitness Center,Ethiopian Restaurant,Grocery Store,Park
9,Yorkville,Italian Restaurant,Gym,Bar,Coffee Shop,Deli / Bodega,Wine Shop,Pizza Place,Japanese Restaurant,Sushi Restaurant,Diner
12,Upper West Side,Italian Restaurant,Bar,Bakery,Wine Bar,Indian Restaurant,Dessert Shop,Café,French Restaurant,Bagel Shop,Thai Restaurant
19,East Village,Bar,Mexican Restaurant,Cocktail Bar,Ice Cream Shop,Pizza Place,Coffee Shop,Vegetarian / Vegan Restaurant,Korean Restaurant,Wine Bar,Vietnamese Restaurant
20,Lower East Side,Chinese Restaurant,Coffee Shop,Japanese Restaurant,Ramen Restaurant,Art Gallery,Café,Bakery,Sandwich Place,Filipino Restaurant,Flower Shop
25,Manhattan Valley,Coffee Shop,Bar,Pizza Place,Mexican Restaurant,Thai Restaurant,Playground,Indian Restaurant,Cosmetics Shop,French Restaurant,Malay Restaurant
27,Gramercy,Bar,Bagel Shop,Italian Restaurant,American Restaurant,Pizza Place,Cocktail Bar,Coffee Shop,Playground,Thrift / Vintage Store,Mexican Restaurant


**Cluster 2**

In [21]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Discount Store,Gym,Sandwich Place,Coffee Shop,Video Game Store,Tennis Stadium,Seafood Restaurant,Donut Shop,Supplement Shop,Pharmacy
15,Midtown,Hotel,Clothing Store,Coffee Shop,Steakhouse,Sporting Goods Shop,Theater,Bookstore,Sandwich Place,Cocktail Bar,Cosmetics Shop
16,Murray Hill,Hotel,Sandwich Place,Coffee Shop,American Restaurant,Bar,Burger Joint,Japanese Restaurant,Italian Restaurant,Gym / Fitness Center,Vietnamese Restaurant
28,Battery Park City,Park,Coffee Shop,Hotel,Memorial Site,Clothing Store,Gym,Beer Garden,Shopping Mall,Burger Joint,Boat or Ferry


**Cluster 3**

In [22]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Italian Restaurant,Coffee Shop,Bakery,Exhibit,Gym / Fitness Center,Yoga Studio,Juice Bar,French Restaurant,Pizza Place,Spa
10,Lenox Hill,Italian Restaurant,Pizza Place,Sushi Restaurant,Coffee Shop,Cocktail Bar,Burger Joint,Gym / Fitness Center,Gym,Café,Deli / Bodega
13,Lincoln Square,Café,Plaza,Italian Restaurant,Gym / Fitness Center,Concert Hall,Performing Arts Venue,Theater,Indie Movie Theater,French Restaurant,Wine Shop
14,Clinton,Theater,Gym / Fitness Center,Gym,American Restaurant,Coffee Shop,Italian Restaurant,Spa,Wine Shop,Sandwich Place,Hotel
17,Chelsea,Coffee Shop,Bakery,Art Gallery,American Restaurant,Hotel,Wine Shop,Italian Restaurant,French Restaurant,Cupcake Shop,Cycle Studio
18,Greenwich Village,Italian Restaurant,Clothing Store,Sushi Restaurant,Café,American Restaurant,French Restaurant,Dessert Shop,Bubble Tea Shop,Indian Restaurant,Seafood Restaurant
21,Tribeca,Park,Italian Restaurant,Spa,American Restaurant,Greek Restaurant,Wine Bar,Bakery,Café,Coffee Shop,Men's Store
22,Little Italy,Bakery,Italian Restaurant,Bubble Tea Shop,Café,Hotel,Sandwich Place,Salon / Barbershop,Pizza Place,Mediterranean Restaurant,Cocktail Bar
23,Soho,Clothing Store,Italian Restaurant,Coffee Shop,Mediterranean Restaurant,Sporting Goods Shop,Bakery,Women's Store,Boutique,Salon / Barbershop,Asian Restaurant
24,West Village,Italian Restaurant,Jazz Club,New American Restaurant,Cocktail Bar,Wine Bar,Park,Coffee Shop,Cosmetics Shop,Theater,American Restaurant


**Cluster 4**

In [23]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Bakery,Bank,Mobile Phone Shop,Grocery Store,Wine Shop,Supermarket,Sandwich Place,Park,Chinese Restaurant
3,Inwood,Mexican Restaurant,Restaurant,Café,Lounge,Frozen Yogurt Shop,Bakery,Pizza Place,Park,Chinese Restaurant,Caribbean Restaurant
4,Hamilton Heights,Pizza Place,Mexican Restaurant,Deli / Bodega,Coffee Shop,Café,Yoga Studio,Bakery,Latin American Restaurant,Park,Cocktail Bar
5,Manhattanville,Coffee Shop,Seafood Restaurant,Italian Restaurant,Mexican Restaurant,Chinese Restaurant,Sushi Restaurant,Boutique,Supermarket,Gastropub,Spanish Restaurant
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Deli / Bodega,Donut Shop,Sandwich Place,Latin American Restaurant,Spa,Pizza Place,Grocery Store
11,Roosevelt Island,Park,Cosmetics Shop,Outdoors & Recreation,Greek Restaurant,Metro Station,Supermarket,Playground,Bubble Tea Shop,Bus Line,Food & Drink Shop
26,Morningside Heights,Coffee Shop,Park,American Restaurant,Bookstore,Café,Deli / Bodega,Burger Joint,Farmers Market,Mediterranean Restaurant,Salad Place
36,Tudor City,Café,Park,Mexican Restaurant,Diner,Pizza Place,Deli / Bodega,Coffee Shop,Seafood Restaurant,Thai Restaurant,Sushi Restaurant


**Cluster 5**

In [24]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Bar,Park,Boat or Ferry,Coffee Shop,Bistro,Gym / Fitness Center,Heliport,Farmers Market,Cocktail Bar,Skating Rink
